In [ ]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import xarray as xr
from rich import traceback

traceback.install(show_locals=True)

xr.set_options(display_expand_attrs=False)

labels_dir = Path("/home/pd/tohoel001/repositories/ML_training_labels/retrogressive_thaw_slumps")

In [ ]:
footprints = (gpd.read_file(footprints_file) for footprints_file in labels_dir.glob("*/ImageFootprints*.gpkg"))
footprints = gpd.GeoDataFrame(pd.concat(footprints, ignore_index=True))
footprints

In [ ]:
def parse_date(row):
    orthotile = row["datasource"] == "PlanetScope OrthoTile"
    if orthotile:
        return pd.to_datetime(row["image_id"].split("_")[-2], format="%Y-%m-%d")
    else:
        return pd.to_datetime(row["image_id"].split("_")[0], format="%Y%m%d")


footprints["date"] = footprints.apply(parse_date, axis=1)
footprints.info()

In [ ]:
from darts_acquisition.s2 import match_s2ids_from_geodataframe_stac

matches = match_s2ids_from_geodataframe_stac(
    aoi=footprints.head(5),
    day_range=7,
    max_cloud_cover=10,
    simplify_geometry=0.1,
)
matches

In [ ]:
for s2id, items in matches.items():
    print(f"{s2id}: {len(items)} items")

In [ ]:
from darts.utils.copernicus import init_copernicus

init_copernicus()

In [ ]:
from darts_acquisition.s2 import load_s2_from_stac

s2item = next(iter(next(iter(matches.values())).values()))  # Get the first item from the first match
s2item

In [ ]:
s2ds = load_s2_from_stac(s2item)
s2ds

In [ ]:
s2ds.quality_data_mask[::20, ::20].plot()

In [ ]:
s2ds.green[::20, ::20].plot()

In [ ]:
s2ds.scl[::20, ::20].plot()